# Demo 2: ATEK Data Store and Model Inference

In [ ]:
import faulthandler

import logging
import os
from logging import StreamHandler
import numpy as np
from typing import Dict, List, Optional
import torch
import sys
import subprocess
from itertools import islice
from tqdm import tqdm

from atek.viz.atek_visualizer import NativeAtekSampleVisualizer
from atek.data_loaders.atek_wds_dataloader import (
    create_native_atek_dataloader
)
from atek.data_loaders.cubercnn_model_adaptor import (
    cubercnn_collation_fn,
    create_atek_dataloader_as_cubercnn
)
from atek.util.file_io_utils import load_yaml_and_extract_tar_list

from cubercnn.config import get_cfg_defaults
from cubercnn.modeling.backbone import build_dla_from_vision_fpn_backbone  # noqa
from cubercnn.modeling.meta_arch import build_model  # noqa
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from omegaconf import OmegaConf

faulthandler.enable()

# Configure logging to display the log messages in the notebook
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)

logger = logging.getLogger()


# -------------------- Helper functions --------------------#
def run_command_and_display_output(command):
    # Start the process
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

    # Poll process.stdout to show stdout live
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())
    rc = process.poll()
    return rc

def create_inference_model(config_file, ckpt_dir, use_cpu_only=False):
    """
    Create the model for inference pipeline, with the model config.
    """
    # Create default model configuration
    model_config = get_cfg()
    get_cfg_defaults(model_config)

    # add extra configs for data
    model_config.MAX_TRAINING_ATTEMPTS = 3
    model_config.TRAIN_LIST = ""
    model_config.TEST_LIST = ""
    model_config.TRAIN_WDS_DIR = ""
    model_config.TEST_WDS_DIR = ""
    model_config.ID_MAP_JSON = ""
    model_config.OBJ_PROP_JSON = ""
    model_config.CATEGORY_JSON = ""
    model_config.DATASETS.OBJECT_DETECTION_MODE = ""
    model_config.SOLVER.VAL_MAX_ITER = 0
    model_config.SOLVER.MAX_EPOCH = 0

    model_config.merge_from_file(config_file)
    if use_cpu_only:
        model_config.MODEL.DEVICE = "cpu"
    model_config.freeze()

    model = build_model(model_config, priors=None)

    _ = DetectionCheckpointer(model, save_dir=ckpt_dir).resume_or_load(
        model_config.MODEL.WEIGHTS, resume=True
    )
    model.eval()

    return model_config, model

### Setup data and code path

In [ ]:
data_dir = "/home/louy/Calibration_data_link/Atek/2024_08_05_DryRun"
atek_src_path = os.path.join(os.path.expanduser("~"), "atek_on_fbsource")
data_store_viz_conf = OmegaConf.load(os.path.join(data_dir, "data_store_viz_conf.yaml"))
infer_viz_conf = OmegaConf.load(os.path.join(data_dir, "infer_viz_conf.yaml"))

# Set up trained model weight path
model_ckpt_path = "/home/louy/Calibration_data_link/Atek/pre_trained_models/2024_08_28_AdtCubercnnWeights"

## Part 1: ATEK Data Store
**Before ATEK**: user needs to set up their own preprocessing, and run through large datasets. **time- & resource- consuming** (14 days for 10K seqs). 

**With ATEK**: preprocessed Aria datasets hosted on **ATEK Data Store**:  
1. [Option 1] Download to local.
2. [Option 2] Stream over internet.

Streaming example below: 

In [ ]:
# First, download json file from ATEK Data Store
atek_json_path = os.path.join(data_dir, "AriaDigitalTwin_ATEK_download_urls.json")
if not os.path.exists(atek_json_path):
    logger.error("Please download AriaDigitalTwin_ATEK_download_urls.json from ATEK Data Store")
    exit()

# Second, parse into streamable yaml files
create_streamable_yaml_command = [
    "python3", f"{atek_src_path}/tools/dataverse_url_parser.py",
    "--config-name","cubercnn",
    "--input-json-path",f"{atek_json_path}",
    "--output-folder-path",f"{data_dir}/streamable_yamls/",
    "--max-num-sequences", "5",
    "--train-val-split-ratio", "0.8"
]
return_code = run_command_and_display_output(create_streamable_yaml_command)

logger.info(f"Streamable yaml files created under f{data_dir}/streamable_yamls/")

# Part 2: Run Object detection inference using pre-trained CubeRCNN model
In this example, we demonstrate how to run model inference with preprocessed ATEK data streamed from Data Store. 

### Create a PyTorch DataLoader from ATEK WDS files
Load WDS files as ATEK format: 

In [ ]:
# create ATEK dataloader with native ATEK format.
tar_file_urls = load_yaml_and_extract_tar_list(yaml_path = os.path.join(data_dir, "streamable_yamls", "streamable_validation_tars.yaml"))
# tar_file_urls = load_yaml_and_extract_tar_list(yaml_path = os.path.join(data_dir, "downloaded_local_wds_adt", "local_all_tars.yaml"))

atek_dataloader = create_native_atek_dataloader(urls = tar_file_urls, batch_size = None, num_workers = 1)
first_atek_sample = next(iter(atek_dataloader))
logger.info(f"Loading WDS into ATEK native format, each sample contains the following keys: {first_atek_sample.keys()}")

### Create PyTorch DataLoader, converted to CubeRCNN format
User can add a data transform function from ATEK format -> CubeRCNN format: 
1. Dict key remapping.
2. Tensor reshaping & reordering. 
3. Other data transformations.

Example data transform function for CubeRCNN model: [src code](https://www.internalfb.com/code/fbsource/[a5c3831c045bc718862d1c512e84d4ed6f79d722]/fbcode/surreal/data_services/atek/atek/data_loaders/cubercnn_model_adaptor.py?lines=44-74)

In [ ]:
cubercnn_dataloader = create_atek_dataloader_as_cubercnn(urls = tar_file_urls, batch_size = 6, num_workers = 1)
first_cubercnn_sample = next(iter(cubercnn_dataloader))
logger.info(f"Loading WDS into CubeRCNN format, each sample contains the following keys: {first_cubercnn_sample[0].keys()}")

### Run model inference

In [ ]:
from tqdm import tqdm

# load pre-trained CubeRCNN model
model_config_file = os.path.join(model_ckpt_path, "config.yaml")
conf = OmegaConf.load(model_config_file)

# setup config and model
model_config, model = create_inference_model(
    model_config_file, model_ckpt_path, False
)


# Cache inference results for visualization
input_output_data_pairs = []

# Loop over created Pytorch Dataloader, only 5 batches for demonstration
with torch.no_grad():
    for cubercnn_input_data in tqdm(
       islice(cubercnn_dataloader, 5),
        desc="Inference progress: ",
    ):
        cubercnn_model_output = model(cubercnn_input_data)

        # cache inference results for visualization
        input_output_data_pairs.append((cubercnn_input_data, cubercnn_model_output))

logger.info("Inference completed.")

### Visualize inference results

In [ ]:
from atek.viz.cubercnn_visualizer import CubercnnVisualizer

# Visualize cached inference results
logger.info("Visualizing inference results.")
cubercnn_visualizer = CubercnnVisualizer(viz_prefix = "inference_visualizer", conf = infer_viz_conf)
for input_data_as_list, output_data_as_list in input_output_data_pairs:
    for single_cubercnn_input, single_cubercnn_output in zip(input_data_as_list, output_data_as_list):
        timestamp_ns = single_cubercnn_input["timestamp_ns"]
        # Plot RGB image
        cubercnn_visualizer.plot_cubercnn_img(single_cubercnn_input["image"], timestamp_ns = timestamp_ns)

        # Plot GT and prediction in different colors
        single_cubercnn_output["T_world_camera"] = single_cubercnn_input["T_world_camera"] # This patch is needed for visualization
        cubercnn_visualizer.plot_cubercnn_dict(cubercnn_dict = single_cubercnn_input, timestamp_ns = timestamp_ns, plot_color = cubercnn_visualizer.COLOR_GREEN, suffix = "_model_input")
        cubercnn_visualizer.plot_cubercnn_dict(cubercnn_dict = single_cubercnn_output, timestamp_ns = timestamp_ns, plot_color = cubercnn_visualizer.COLOR_RED, suffix = "_model_output")

## Part 3: Evaluate model performance
ATEK provides **per-task**: 
1. Standardized prediction file formats.
2. Lib for common eval metrics.
3. Benchmarking scripts.

Example prediction file format for 3D object detection: 

| time_ns       | tx_world_object | ty_world_object | tz_world_object | qw_world_object | qx_world_object | qy_world_object | qz_world_object | scale_x | scale_y | scale_z | name    | instance | sem_id | prob    |
|---------------|-----------------|-----------------|-----------------|-----------------|-----------------|-----------------|-----------------|---------|---------|---------|---------|----------|--------|--------|
| 14588033546600| -4.119894       | 0.986124        | 2.796770        | 0.008052        | -0.022706       | -0.010150       | 0.999658        | 0.190   | 2.146   | 0.900   | door    | -1       | 32     | 0.994462|
| 14588033546600| -3.875954       | 0.837941        | 4.056602        | 0.009215        | -0.015670       | 0.999661        | -0.018645       | 0.325   | 1.697   | 0.964   | display | -1       | 37     | 0.994381|

### Write inference results into ATEK-format csv files


In [ ]:
from atek.evaluation.static_object_detection.obb3_csv_io import AtekObb3CsvWriter

gt_writer = AtekObb3CsvWriter(output_filename = os.path.join(data_dir, "gt_obbs.csv"))
prediction_writer = AtekObb3CsvWriter(output_filename = os.path.join(data_dir, "prediction_obbs.csv"))

for input_data_as_list, output_data_as_list in input_output_data_pairs:
    for single_cubercnn_input, single_cubercnn_output in zip(input_data_as_list, output_data_as_list):
        timestamp_ns = single_cubercnn_input["timestamp_ns"]
        single_cubercnn_output["T_world_camera"] = single_cubercnn_input["T_world_camera"]

        gt_writer.write_from_cubercnn_dict(cubercnn_dict = single_cubercnn_input, timestamp_ns = timestamp_ns)
        prediction_writer.write_from_cubercnn_dict(cubercnn_dict = single_cubercnn_output, timestamp_ns = timestamp_ns)

### Call ATEK's benchmarking script to evaluate the results

In [ ]:
benchmarking_command = [
    "python3", f"{atek_src_path}/tools/benchmarking_static_object_detection.py",
    "--pred-csv", f"{data_dir}/prediction_obbs.csv",
    "--gt-csv", f"{data_dir}/gt_obbs.csv",
    "--output-file", f"{data_dir}/atek_metrics.json"
]
return_code = run_command_and_display_output(benchmarking_command)